# Testing EBM with a simple example

Parameters:

In [1]:
experiment = 'mid_datab'
arch = 'fhi360_large_1_23_convnet'
ckpt_experiment = 'mid_data_1' 
IMG_SHAPE = (454, 454,3)
NUM_CLASSES = 23
seed=13456
labels = ['Albendazole','Amoxicillin','Ampicillin','Azithromycin','Benzyl-Penicillin','Blank','Ceftriaxone','Chloroquine','Ciprofloxacin','Doxycycline','Epinephrine', \
          'Ethambutol','Ferrous-Sulfate','Hydroxychloroquine','Isoniazid','Lactose', \
          'Promethazine-Hydrochloride','Pyrazinamide','Rifampicin','Ripe','Sulfamethoxazole', \
          'Swiped-but-not-run','Tetracycline']
weights = 'fhi360_large_1_23_convnetmid_datab.tar.gz'
MODEL_DIR = '../../models/weights/fhi360_large_1_23_convnetmid_datab'
GPU_ID = '1'


### Specify a particular GPU for TensorFlow to use

In [2]:
import os

# Ensure CUDA_VISIBLE_DEVICES is set BEFORE TensorFlow is loaded
os.environ['CUDA_VISIBLE_DEVICES'] = GPU_ID

# Import TensorFlow after setting the environment variable
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

# Explicitly set which GPU devices are visible to TensorFlow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Attempt to set GPU 1 as the only visible device
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

# Check GPU availability again
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print('Visible Devices:', tf.config.get_visible_devices('GPU'))

2024-04-26 14:06:04.701282: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 14:06:04.753871: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 14:06:04.753905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 14:06:04.753938: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 14:06:04.763004: I tensorflow/core/platform/cpu_feature_g

2.14.0
Num GPUs Available:  1
Visible Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Test trained network

In [4]:
import os
from PIL import Image, ImageEnhance, ImageStat
import tensorflow as tf
from tensorflow import keras
import pandas as pd  
import numpy as np

# # diagnostics
predict = np.zeros(NUM_CLASSES)
total = np.zeros(NUM_CLASSES)
predmat = np.zeros((NUM_CLASSES, NUM_CLASSES))

# calibration storage
start=0.05
step=.05
num=21

bins=np.arange(0,num)*step+start+ 1e-10
bin_total = np.zeros(20)+1e-5
bin_correct = np.zeros(20)

total_correct = 0.
total_total = 0

energies = []

# test trained network

model_dir = MODEL_DIR 

reconstructed_model = keras.models.load_model(model_dir, compile=False)

labels = ['Albendazole','Amoxicillin','Ampicillin','Azithromycin','Benzyl-Penicillin','Blank','Ceftriaxone','Chloroquine','Ciprofloxacin','Doxycycline','Epinephrine', \
          'Ethambutol','Ferrous-Sulfate','Hydroxychloroquine','Isoniazid','Lactose','Promethazine-Hydrochloride','Pyrazinamide','Rifampicin','Ripe','Sulfamethoxazole','Swiped-but-not-run','Tetracycline']

# force to lower for finding index
for i in range(len(labels)):
    labels[i] = labels[i].lower()

root = '/mnt/slow_data/pmoreira/datasets/FHI2020_Stratified_Sampling'
csvfile = os.path.join(root, 'test_labels.csv')

image_folder = os.path.join(root, 'test_images')

count = 0

2024-04-26 08:15:19.423793: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46772 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b3:00.0, compute capability: 8.6
